<h1 align="center"> Sensordatenfusion Tutorium 04 </h1>

<h3 align="center"> Messmodel und Updater </h3>


Ziel dieses Tutoriums ist es, $x_{k|k}$ und $P_{k|k}$ zu erhalten, also den Filter-Schritt durchzuführen.
Dazu benötigen wir noch die Messmatrix $H_k$ und ein Rauschen $R_k$.

$H$ = $\begin{bmatrix}
  1 & 0 & 0 & 0 \\
  0 & 0 & 1 & 0  \\
\end{bmatrix}$

$R$ = $\begin{bmatrix}
  \sigma² & 0  \\
  0 & \sigma²  \\
\end{bmatrix}$
wobei $\sigma$ = 50

In [1]:
import numpy as np

from stonesoup.types.array import CovarianceMatrix

from scipy.stats import multivariate_normal

from stonesoup.models.base import LinearModel, GaussianModel
from stonesoup.models.measurement.base import MeasurementModel

<h3 align="center"> Messmodell erstellen </h3>

In [3]:
class SDFMessmodell(MeasurementModel, LinearModel, GaussianModel):
    
    @property
    def ndim_meas(self):
        return 2
    
    def matrix(self, **kwargs):
        model_matrix = """Matrix H"""
        return model_matrix
    
    def covar(self):
        noise_matrix = """Matrix R"""
        return CovarianceMatrix(noise_matrix)
    
    def rvs(self):
        # sample ziehen aus der Kovarianzmatrix
        noise = multivariate_normal.rvs(np.zeros(self.ndim_meas), self.covar(), 1)
        return noise.reshape((-1, 1))
    
    def pdf(self):
        pass
    
    
    

In [4]:
messmodell = SDFMessmodell(4, (0, 2))   # Messmodell initiieren

In [5]:
messmodell.matrix() # zum testen ob eure Matrix richtig definiert ist

'Matrix H'

In [6]:
messmodell.covar()  # zum testen ob eure Matrix richtig definiert ist


'Matrix R'

<h4 align="center"> Imports für den Updater </h3>

In [7]:
from stonesoup.updater.base import Updater
from stonesoup.types.hypothesis import SingleHypothesis
from stonesoup.types.prediction import GaussianMeasurementPrediction
from stonesoup.types.update import GaussianStateUpdate

<h3 align="center"> Updater erstellen </h3>

In [8]:
class SDFUpdater(Updater):

    def get_measurement_prediction(self, state_prediction, measurement_model=None, **kwargs):
        x_pre = state_prediction
        Messmatrix = measurement_model.matrix()
        messprediction = """ H @ xk|k-1 """
        return messprediction


    def update(self, hypothesis, measurementmodel, **kwargs):
        
        measurement_matrix = measurementmodel.matrix()  # H
        measurement_noise_covar = measurementmodel.covar()  # R
        prediction_covar = hypothesis.prediction.covar  # P
        prediction_mean = hypothesis.prediction.mean    # x_k|k-1
        measurement_mean = hypothesis.measurement.state_vector  # z_k
        messprediction = self.get_measurement_prediction(prediction_mean, measurementmodel)

        """ v (Innovation), S und W  definieren"""
        
        x_post = """ x_k|k  berechnen"""
        P_post = """ P_k|k  berechnen"""

        hypothesis = SingleHypothesis(hypothesis.prediction,
                                      hypothesis.measurement,
                                      GaussianMeasurementPrediction(
                                          messprediction, S,
                                          hypothesis.prediction.timestamp)
                                      )

        return GaussianStateUpdate(x_post,
                                   P_post,
                                   hypothesis,
                                   hypothesis.measurement.timestamp)